In [1]:
import pandas as pd
filepath_dict = {'train':'C:/Users/kelly/OneDrive/Desktop/R Code/train_data.csv',
                 'submission':'C:/Users/kelly/OneDrive/Desktop/R Code/test.csv'
                }
df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath)
    df['source'] = source  # Add another column filled with the source name
    df_list.append(df)
    
df = pd.concat(df_list)
print(df.iloc[2])

Category                           2
itemid                       3592295
source                         train
title       milani rise powder blush
Name: 2, dtype: object


C:\Users\kelly\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # This is added back by InteractiveShellApp.init_path()


In [3]:
# Create a vocabulary in the case is a list of words that ocurred in our text where each word has its own index
# Vectorise a sentence and count the number of occurance 
# Feature vector, each dimension can be numeric or categorical feature, like for example the height of a bulding

from sklearn.feature_extraction.text import CountVectorizer
sentences = ['John likes ice cream', 'John hates chocolate.']
vectorizer = CountVectorizer(min_df=0, lowercase=False)
vectorizer.fit(sentences)
vectorizer.vocabulary_

{'John': 0, 'likes': 5, 'ice': 4, 'cream': 2, 'hates': 3, 'chocolate': 1}

In [4]:
# Create document term array
# This is considered a bag of word model 
vectorizer.transform(sentences).toarray()

array([[1, 0, 1, 0, 1, 1],
       [1, 1, 0, 1, 0, 0]], dtype=int64)

In [2]:
# split the data into a training and testing set
from sklearn.model_selection import train_test_split

df_yelp = df[df['source'] == 'train']
sentences = df_yelp['title'].values
# fixed_sentences = v.fit_transform(df_yelp['title'].values.astype('U'))

y = df_yelp['Category'].values

sentence_train,sentence_test,y_train,y_test = train_test_split(sentences,y,test_size=0.25,random_state=1000)


(499961,)
(166654,)


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
x_train = []
x_test = []

vectorizer = TfidfVectorizer(max_df = 0.5, min_df = 2,ngram_range=(1, 3),encoding='utf-8',strip_accents='unicode')
vectorizer.fit(sentence_train.astype('U'))

x_train = vectorizer.transform(sentence_train.astype('U'))
x_test = vectorizer.transform(sentence_test.astype('U'))
x_train

<499961x446067 sparse matrix of type '<class 'numpy.float64'>'
	with 9829584 stored elements in Compressed Sparse Row format>

In [4]:
# Baseline Model - logistic Regression
from sklearn.linear_model import LogisticRegression

logistic_classifier = LogisticRegression()

logistic_classifier.fit(x_train, y_train)

logistic_label = logistic_classifier.predict(x_test)

score = logistic_classifier.score(x_test, y_test)

print("Accuracy:", score)

Accuracy: 0.7328236946007897


In [36]:
from sklearn.metrics import confusion_matrix
# Create a confusion matrix
logistic_confusion_m = confusion_matrix(y_test,logistic_label)

prediction = pd.DataFrame(logistic_confusion_m).to_csv('Logistic_confusion_m.csv')

logistic_confusion_m

array([[ 375,   38,   54, ...,    0,    0,    0],
       [  11, 4463,   46, ...,    0,    0,    0],
       [  27,   33, 2442, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ...,   25,    0,    0],
       [   0,    0,    0, ...,    0,    8,    0],
       [   0,    0,    0, ...,    0,    0,    0]], dtype=int64)

In [38]:
logistic_pred = logistic_classifier.predict(x_test)
logistic_df = {'test_case': sentence_test,
               'prediction':logistic_pred,
               'Actual':y_test}

logistic_df = pd.DataFrame(logistic_df)
logistic_export = logistic_df.to_csv("C:/Users/kelly/OneDrive/Desktop/R Code/Shopee Predictionlogistic_reg_prediction.csv",index = None, header = True)

In [49]:
# train classifer using the whole dataset
shopee_train_data = df[df['source'] == 'train']
shopee_train_sentences = shopee_train_data['title'].values
shopee_train_y = shopee_train_data['Category'].values

shopee_test_data = df[df['source'] == 'submission']
shopee_test_sentences = shopee_test_data['title'].values

vectorizer.fit(shopee_train_sentences.astype('U'))
# Vectorised dataset for all training set
train_sentences = vectorizer.transform(shopee_train_sentences.astype('U'))
# Vectorised dataset for submission data set
test_sentences = vectorizer.transform(shopee_test_sentences.astype('U'))

In [ ]:
shopee_logistic_classifier = LogisticRegression()
shopee_logistic_classifier.fit(train_sentences, shopee_train_y)
shopee_logistic_label = shopee_logistic_classifier.predict(test_sentences)
# Print out the prediction 
logistic_df = {'test_case': shopee_test_sentences,
               'prediction':shopee_logistic_label,
              }
submission_df = pd.DataFrame(logistic_df)
logistic_export = submission_df.to_csv("C:/Users/kelly/OneDrive/Desktop/R Code/Shopee Prediction/logistic_submission_prediction.csv",index = None, header = True)

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.preprocessing import label_binarize

df_yelp = df[df['source'] == 'train']

# fixed_sentences = v.fit_transform(df_yelp['title'].values.astype('U'))

y = df_yelp['Category'].values

# y = label_binarize(y,np.asarray(range(58)))

sentence_train,sentence_test,y_train,y_test = train_test_split(sentences,y,test_size=0.35,random_state=1000,stratify=y)

x_train = []
x_test = []

vectorizer = TfidfVectorizer(max_df = 0.5, min_df = 2,ngram_range=(1, 3),encoding='utf-8',strip_accents='unicode')
vectorizer.fit(sentence_train.astype('U'))

x_train = vectorizer.transform(sentence_train.astype('U'))
x_test = vectorizer.transform(sentence_test.astype('U'))

In [17]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import roc_auc_score
import parfit.parfit as pf

from sklearn.multiclass import OneVsRestClassifier

# Improve SVM
# Grid tuning

clf_SVM = OneVsRestClassifier(LinearSVC(class_weight="balanced"))

grid = {
     'alpha': [1e-6, 1e-4, 1e-1, 1e1, 1e3], # learning rate
     'n_iter': [100], # number of epochs
     'loss': ['hinge'], # logistic regression,
     'penalty': ['l2'],
     'n_jobs': [-1]
}
paramGrid = ParameterGrid(grid)

best_model, best_score, all_models, all_scores = pf.bestFit(clf_SVM, paramGrid, 
      x_train, y_train, x_test,y_test, 
      metric=roc_auc_score, scoreLabel='AUC')

print(best_model, best_score)


KeyboardInterrupt: 

In [41]:
# Create a svm model
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
# During training, we can use the argument class_weight = "balanced" to penalize mistakes on the minority class n
# by an amount proportional to how under represented it is
# alpha = 1e-3: 0.663606, iter = 5
# alpha = 1e-4: 0.682554, iter = 5
# alpha = 1e-4: 0.701756, iter = 100
# alpha = 1e-5: 0.609156, iter = 5
# alpha = 1e-4: ,iter = 1000

clf_svm = OneVsRestClassifier(SGDClassifier(loss='hinge', penalty='l2',alpha=1e-4,class_weight="balanced", random_state=1000, max_iter=1000, tol=None))
clf_svm.fit(x_train,y_train)

y_pred = clf_svm.predict(x_test)

In [44]:
# Predict using SVM
score = clf_svm.score(x_test, y_test)

svm_prediction = {'title': sentence_test,
                  'prediction': y_pred
                 }

svm_df = pd.DataFrame(svm_prediction)
svm_export = svm_df.to_csv("C:/Users/kelly/OneDrive/Desktop/R Code/Shopee Prediction/svm_test_prediction.csv",index = None, header = True)

print("SVM Accuracy:", score)

SVM Accuracy: 0.7018335647790979


In [ ]:
# Predicting Shopee submission data 
shopee_svm_classifier = svclassifier
shopee_svm_classifier.fit(train_sentences, shopee_train_y)
shopee_svm_label = shopee_svm_classifier.predict(test_sentences)

svm_df = {'test_case': shopee_test_sentences,
          'prediction':shopee_svm_label
          }

final_test_result = {'itemid':shopee_test_data['itemid'].values,
                     'Category':shopee_svm_label
                    }

submission_df = pd.DataFrame(svm_df)
final_test_df = pd.DataFrame(final_test_result)
svm_export = submission_df.to_csv("C:/Users/kelly/OneDrive/Desktop/R Code/Shopee Prediction/svm_submission_prediction.csv",index = None, header = True)
svm_submission = final_test_df.to_csv("C:/Users/kelly/OneDrive/Desktop/R Code/Shopee Prediction/final_test_result_7.csv",index = None, header = True)

In [59]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 1000)

rf_clf = RandomForestClassifier(random_state = 1000, class_weight="balanced")

scoring = {'Accuracy': make_scorer(accuracy_score),
           'Precision': make_scorer(precision_score)
}

params = { 'max_depth': [50,100,150],
          'min_samples_split':[5],
          'min_samples_leaf':[8],
          
          'n_estimators':[1000,2000],
          'max_features':['auto','sqrt']
}

#grid_rf_clf = GridSearchCV(rf_clf, param_grid = params, cv= cv, n_jobs=-1, verbose = 4)
#grid_rf_clf.fit(x_train,y_train)


In [55]:
print(grid_rf_clf.best_estimator_)
print(grid_rf_clf.best_params_)
print(grid_rf_clf.best_score_)
s = grid_rf_clf.score(x_test,y_test)
print("Accuracy:",s)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=20, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=8,
            min_samples_split=5, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=False, random_state=1000,
            verbose=0, warm_start=False)
{'max_depth': 20, 'min_samples_leaf': 8, 'min_samples_split': 5, 'n_estimators': 500}
0.5601466885453232
Accuracy: 0.5647276654837217


In [56]:
# increase max_depth by 5
rf_clf_depth_25 = RandomForestClassifier(random_state = 1000,max_depth = 25, min_samples_split=5,min_samples_leaf=8,n_estimators=500,class_weight="balanced")
rf_clf_depth_25.fit(x_train,y_train)
s2 = rf_clf_depth_25.score(x_test,y_test)
print("Accuracy:",s2)

Accuracy: 0.5683622211935744


In [57]:
# increase n_estimator by 100
rf_clf_estimator_600 = RandomForestClassifier(random_state = 1000,max_depth = 20, min_samples_split=5,min_samples_leaf=8,n_estimators=600,class_weight="balanced")
rf_clf_estimator_600.fit(x_train,y_train)
s3 = rf_clf_estimator_600.score(x_test,y_test)
print("Accuracy:",s3)

Accuracy: 0.5674450102007578


In [60]:
# max_depth = 100
# n estimators = 1000
rf_clf_estimator_1000 = RandomForestClassifier(random_state = 1000,max_depth = 100, min_samples_split=5,min_samples_leaf=8,n_estimators=1000,class_weight="balanced")
rf_clf_estimator_1000.fit(x_train,y_train)
s4 = rf_clf_estimator_1000.score(x_test,y_test)
print("Accuracy:",s4)

Accuracy: 0.5924882991307926


In [86]:
# pre-processing the data
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import ToktokTokenizer
import re

lemma = WordNetLemmatizer()
token = ToktokTokenizer()

stopWordList=stopwords.words('english')

def removeCharDigit(text):
    str="`1234567890-=~@#$%^&*()_+[!{;”:\’><.,/?”}]"
    for w in text:
        if w in str:
            text = text.replace(w,"")
    return text

def lemitizeWords(text):
    words= token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w,"v")
        listLemma.append(x)
    return text

def stopWordsRemove(text):
    wordList=[x.lower().strip() for x in  token.tokenize(text)]
    removedList=[x for x in wordList if not x in stopWordList]
    text=" ".join(removedList)
    return text

def PreProcessing(text):
    text = removeCharDigit(text)
    text = lemitizeWords(text)
    text = stopWordsRemove(text)
    return(text)

df_yelp = df[df['source'] == 'train']

df_yelp = df_yelp.dropna()

sentences = df_yelp['title'].values

sentence_df = pd.DataFrame({'title':sentences})

df_yelp_clean = sentence_df['title'].apply(lambda com:PreProcessing(com))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kelly\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kelly\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [90]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier

# Create one-hot encoding 
y = df_yelp['Category'].values
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)

# convert integers to dummy variables 
dummy_y = np_utils.to_categorical(encoded_Y)
sentences = df_yelp_clean
sentence_train,sentence_test,y_train,y_test = train_test_split(sentences,dummy_y,test_size=0.4,random_state=1000,stratify=y)

vectorizer = TfidfVectorizer(max_df = 0.5, min_df = 2,ngram_range=(1, 3),encoding='utf-8',strip_accents='unicode')
vectorizer.fit(sentence_train.astype('U'))

x_train = []
x_test = []

x_train = vectorizer.transform(sentence_train.astype('U'))
x_test = vectorizer.transform(sentence_test.astype('U'))

# Training Gaussian Naive Bayes classifier
GNB_classifier = BinaryRelevance(GaussianNB())
GNB_classifier.fit(x_train, y_train)

predictions = GNB_classifier.predict(x_test)

score = GNB_classifier.score(x_test,y_test)

print("Accuracy:",score)

MemoryError: 

In [4]:
# Keras - deep learning and neural network API which is capable of running on top of tensorflow
# https://realpython.com/python-keras-text-classification/
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras import layers
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

C:\Users\kelly\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
# Modelling multi-class classification problems using neural networks
# Best pratice is to reshape the output attribute from a vector that contains values for each class
# value to be a matrix with a boolean for each class value and whether or not a given instance has the class value or not
# One hot encoding 
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables 
dummy_y = np_utils.to_categorical(encoded_Y)

In [52]:
# Using one hot encoding for the data set
# Output layer must create 57 output values for each class
# The output value with the largest value will be taken as the class predicted by the model
# SoftMax activation function in the output layer, this ensures that the output range is between 0 and 1
# Network uses the efficient Adam gradient descent algorithm 
sentence_train,sentence_test,y_train,y_test = train_test_split(sentences,dummy_y,test_size=0.3,random_state=1000)
x_train = vectorizer.transform(sentence_train.astype('U'))
x_submission = vectorizer.transform(shopee_test_sentences.astype('U'))
x_test = vectorizer.transform(sentence_test.astype('U'))

model = Sequential()
input_dim = x_train.shape[1]
output_dim = dummy_y.shape[1]
# sequential deep learning model with two layers
model.add(layers.Dense(10,input_dim = input_dim, activation = "relu"))
model.add(layers.Dense(output_dim,activation="softmax"))

In [42]:
# Training the model
# Configure the learning process
# This method specifies the optimiser and the loss function
# We can also specify the list of metrics which can be used for evaluation
model.compile(loss = "binary_crossentropy",
              optimizer = "adam",
              metrics = ['accuracy'])

In [29]:
model.summary()
x_train.shape[1]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 10)                4460680   
_________________________________________________________________
dense_17 (Dense)             (None, 58)                638       
Total params: 4,461,318
Trainable params: 4,461,318
Non-trainable params: 0
_________________________________________________________________


446067

In [43]:
# Neural Network is an iterative process 
# Specify the number of iterations you want the model to be training
# completed iterations are commonly called epochs
# 100 epochs to see how the training loss an accuracy are changing after each epoch
# Another parameter you have to your selection is the batch size 
# The batch size is responsible for how many samples we want to use in one epoch
# How many samples are used in one forward/backward pass

keras_seq_model = model.fit(x_train, y_train,
                            epochs = 5,
                            verbose= 1,
                            validation_data =(x_test,y_test),
                            batch_size = 3000)

Train on 466630 samples, validate on 199985 samples
Epoch 1/5
466630/466630 [==============================] - 6022s 13ms/step - loss: 0.0222 - acc: 0.9924 - val_loss: 0.0259 - val_acc: 0.9909
Epoch 2/5
466630/466630 [==============================] - 5541s 12ms/step - loss: 0.0214 - acc: 0.9926 - val_loss: 0.0255 - val_acc: 0.9910
Epoch 3/5
466630/466630 [==============================] - 4576s 10ms/step - loss: 0.0206 - acc: 0.9929 - val_loss: 0.0253 - val_acc: 0.9911
Epoch 4/5
466630/466630 [==============================] - 5061s 11ms/step - loss: 0.0198 - acc: 0.9932 - val_loss: 0.0250 - val_acc: 0.9911
Epoch 5/5
466630/466630 [==============================] - 6560s 14ms/step - loss: 0.0190 - acc: 0.9934 - val_loss: 0.0248 - val_acc: 0.9912


In [45]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights('keras_seq_model.h5')

In [47]:
train_loss, train_accuracy = model.evaluate(x_train, y_train, verbose=1)
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=1)

199985/199985 [==============================] - 671s 3ms/step


In [48]:
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

Test loss: 0.024847183629554116
Test accuracy: 0.9911794213127171


In [ ]:
# Use the model to predict submission data set
shopee_seq_prediction = model.predict(x_submission)


In [57]:
shopee_seq_prediction_2 = model.predict_classes(x_submission)

In [58]:
# Creating the dataframe for prediction
shopee_test_itemid = shopee_test_data['itemid'].values

baseline_seq = {'itemid':shopee_test_itemid,
                'test_case': shopee_test_sentences,
                'prediction':shopee_seq_prediction_2,
                }

baseline_seq_df = pd.DataFrame(baseline_seq)
baseline_seq_export = baseline_seq_df.to_csv("C:/Users/kelly/OneDrive/Desktop/R Code/Shopee Prediction/baseline_seq_prediction.csv",index = None, header = True)
